In [0]:
monthly_longest_dry_day_spell_csv = "https://drive.google.com/file/d/1bwu2EYumrYAKhj7-mWRurdEG86eZxcEH/view?usp=sharing"
output_file = "administrative-district-monthly-longest-dry-stretch.xlsx"

In [0]:
import pandas as pd
from google.colab import files

In [0]:
def data_cleaning_monthly_longest_dry_spell(url, output_file):
    
    old_prefix = "https://drive.google.com/file/d/"
    new_prefix = "https://drive.google.com/uc?authuser=1&id="
    
    old_suffix = "/view?usp=sharing"
    new_suffix = "&export=download"
    
    data = url.replace(old_prefix, new_prefix)
    data = data.replace(old_suffix, new_suffix)
    df = pd.read_csv(data, index_col=0, parse_dates=True)
    
    district_names = df["Dist_name"]
    df_copy = df.reset_index()
    district_index = df_copy.filter(regex="([0-9]\_[0-9])")

    # Mean and standard deviation dataframe cleaning
    dataframe_mean = district_index.filter(like='mean')
    dataframe_stdDev = district_index.filter(like='stdDev')

    dataframe_mean.columns = dataframe_mean.columns.str.split('_cou').str[0]
    dataframe_stdDev.columns = dataframe_stdDev.columns.str.split('_cou').str[0]

    dataframe_mean.set_index(district_names, inplace=True)
    dataframe_stdDev.set_index(district_names, inplace=True)

    dataframe_mean_new_columns_s0 = dataframe_mean.columns.str.split('_').str[0].astype(int) + 1981
    dataframe_mean_new_columns_s1 = dataframe_mean.columns.str.split('_').str[1].astype(int) + 1
    dataframe_mean_new_columns_s0 = dataframe_mean_new_columns_s0.astype('str')
    dataframe_mean_new_columns_s1 = dataframe_mean_new_columns_s1.astype('str')
    dataframe_mean_new_columns = dataframe_mean_new_columns_s0 + "-" + dataframe_mean_new_columns_s1
    dataframe_mean.columns = dataframe_mean_new_columns
    df_mean_trans = dataframe_mean.transpose()
    datetime_df_mean = df_mean_trans.set_index(pd.to_datetime(df_mean_trans.index))

    dataframe_stdDev_new_columns_s0 = dataframe_stdDev.columns.str.split('_').str[0].astype(int) + 1981
    dataframe_stdDev_new_columns_s1 = dataframe_stdDev.columns.str.split('_').str[1].astype(int) + 1
    dataframe_stdDev_new_columns_s0 = dataframe_stdDev_new_columns_s0.astype('str')
    dataframe_stdDev_new_columns_s1 = dataframe_stdDev_new_columns_s1.astype('str')
    dataframe_stdDev_new_columns = dataframe_stdDev_new_columns_s0 + "-" + dataframe_stdDev_new_columns_s1
    dataframe_stdDev.columns = dataframe_stdDev_new_columns
    df_stdDev_trans = dataframe_stdDev.transpose()
    datetime_df_stdDev = df_stdDev_trans.set_index(pd.to_datetime(df_stdDev_trans.index))

    datetime_df_mean.columns = datetime_df_mean.columns + ("_mean_longest_consecutive_dry_stretch")
    datetime_df_stdDev.columns = datetime_df_stdDev.columns + ("_stdDev_longest_consecutive_dry_stretch")
    
    datetime_df_mean.sort_index(inplace=True)
    datetime_df_stdDev.sort_index(inplace=True)

    dfs_mean = datetime_df_mean[datetime_df_mean.index.month.isin(pd.to_numeric(pd.date_range(str(11)+'-1999',
                                                                     str(4+1)+'-2000',
                                                                     freq='M').strftime("%m")).tolist())]
    dfs_stdDev = datetime_df_stdDev[datetime_df_stdDev.index.month.isin(pd.to_numeric(pd.date_range(str(11)+'-1999',
                                                                     str(4+1)+'-2000',
                                                                     freq='M').strftime("%m")).tolist())]

    df_readme = pd.DataFrame(["This is a cleaned data set depicting longest dry stretch per month (days with precipitation < 0.1 mm)",
                                     "zonal statistics across Zambia's 101 administrative districts for",
                                     "the November through April agricultural growing season. Daily precipitation data from UCSB Climate Hazards Group",
                                     "CHIRPS 2.0 were accessed using Google Earth Engine JavaScript API. Raster data are transformed to",
                                     "CSV files during zonal statistics processing (saved in a public-visibility",
                                     "Google Drive folder)."])
    
    # Mean and standard deviation seasonal resampling and saving to disk
    dfsc_mean_NDJFMA = dfs_mean.copy()
    dfsc_stdDev_NDJFMA = dfs_stdDev.copy()
    df_readme_NDJFMA = pd.DataFrame(["This is a cleaned data set depicting longest dry stretch per month (days with precipitation < 0.1 mm)",
                                     "zonal statistics across Zambia's 101 administrative districts for",
                                     "the NDJFMA agricultural growing season. Daily precipitation data from UCSB Climate Hazards Group",
                                     "CHIRPS 2.0 were accessed using Google Earth Engine JavaScript API. Raster data are transformed to",
                                     "CSV files during zonal statistics processing (saved in a public-visibility",
                                     "Google Drive folder). Raw CSV data are cleaned and resampled to multi-monthly metrics using Python 3.",
                                     "Data and code were prepared by Alex Pakalniskis (pakalniskis@email.arizona.edu) during Summer 2019."])
    
    dfsc_mean_NDJFMA = dfsc_mean_NDJFMA.resample("A").mean()
    dfsc_stdDev_NDJFMA = dfsc_stdDev_NDJFMA.resample("A").mean()
    
    dfsc_mean_NDJFMA.index = dfsc_mean_NDJFMA.index.strftime("%Y")
    dfsc_stdDev_NDJFMA.index = dfsc_stdDev_NDJFMA.index.strftime("%Y")
    
    dfsc_mean_NDJFMA.columns = dfsc_mean_NDJFMA.columns + "_NDJFMA"
    dfsc_stdDev_NDJFMA.columns = dfsc_stdDev_NDJFMA.columns + "_NDJFMA"
        
    dfsc_mean_NDJ = dfs_mean.copy()
    dfsc_stdDev_NDJ = dfs_stdDev.copy()
    df_readme_NDJ = pd.DataFrame(["This is a cleaned data set depicting longest dry stretch per month (days with precipitation < 0.1 mm)",
                                     "zonal statistics across Zambia's 101 administrative districts for",
                                     "the NDJ agricultural growing sub-season. Daily precipitation data from UCSB Climate Hazards Group",
                                     "CHIRPS 2.0 were accessed using Google Earth Engine JavaScript API. Raster data are transformed to",
                                     "CSV files during zonal statistics processing (saved in a public-visibility",
                                     "Google Drive folder). Raw CSV data are cleaned and resampled to multi-monthly metrics using Python 3.",
                                     "Data and code were prepared by Alex Pakalniskis (pakalniskis@email.arizona.edu) during Summer 2019."])
    
    re_mean_dfsc_NDJ = dfsc_mean_NDJ.resample("Q-Jan").mean()
    re_stdDev_dfsc_NDJ = dfsc_stdDev_NDJ.resample("Q-Jan").mean()
    
    dfsc_mean_NDJ_crop = re_mean_dfsc_NDJ[re_mean_dfsc_NDJ.index.month.isin([1])]
    dfsc_stdDev_NDJ_crop = re_stdDev_dfsc_NDJ[re_stdDev_dfsc_NDJ.index.month.isin([1])]
    
    dfsc_mean_NDJ_crop.index = dfsc_mean_NDJ_crop.index.strftime("%Y")
    dfsc_stdDev_NDJ_crop.index = dfsc_stdDev_NDJ_crop.index.strftime("%Y")
    
    dfsc_mean_NDJ_crop.columns = dfsc_mean_NDJ_crop.columns + "_NDJ"
    dfsc_stdDev_NDJ_crop.columns = dfsc_stdDev_NDJ_crop.columns + "_NDJ"
            
    dfsc_mean_DJF = dfs_mean.copy()
    dfsc_stdDev_DJF = dfs_stdDev.copy()
    df_readme_DJF = pd.DataFrame(["This is a cleaned data set depicting longest dry stretch per month (days with precipitation < 0.1 mm)",
                                     "zonal statistics across Zambia's 101 administrative districts for",
                                     "the DJF agricultural growing sub-season. Daily precipitation data from UCSB Climate Hazards Group",
                                     "CHIRPS 2.0 were accessed using Google Earth Engine JavaScript API. Raster data are transformed to",
                                     "CSV files during zonal statistics processing (saved in a public-visibility",
                                     "Google Drive folder). Raw CSV data are cleaned and resampled to multi-monthly metrics using Python 3.",
                                     "Data and code were prepared by Alex Pakalniskis (pakalniskis@email.arizona.edu) during Summer 2019."])
    
    re_mean_dfsc_DJF = dfsc_mean_DJF.resample("Q-Feb").mean()
    re_stdDev_dfsc_DJF = dfsc_stdDev_DJF.resample("Q-Feb").mean()
    
    dfsc_mean_DJF_crop = re_mean_dfsc_DJF[re_mean_dfsc_DJF.index.month.isin([2])]
    dfsc_stdDev_DJF_crop = re_stdDev_dfsc_DJF[re_stdDev_dfsc_DJF.index.month.isin([2])]
    
    dfsc_mean_DJF_crop.index = dfsc_mean_DJF_crop.index.strftime("%Y")
    dfsc_stdDev_DJF_crop.index = dfsc_stdDev_DJF_crop.index.strftime("%Y")
    
    dfsc_mean_DJF_crop.columns = dfsc_mean_DJF_crop.columns + "_DJF"
    dfsc_stdDev_DJF_crop.columns = dfsc_stdDev_DJF_crop.columns + "_DJF"
                   
    dfsc_mean_JFM = dfs_mean.copy()
    dfsc_stdDev_JFM = dfs_stdDev.copy()
    df_readme_JFM = pd.DataFrame(["This is a cleaned data set depicting longest dry stretch per month (days with precipitation < 0.1 mm)",
                                     "zonal statistics across Zambia's 101 administrative districts for",
                                     "the JFM agricultural growing sub-season. Daily precipitation data from UCSB Climate Hazards Group",
                                     "CHIRPS 2.0 were accessed using Google Earth Engine JavaScript API. Raster data are transformed to",
                                     "CSV files during zonal statistics processing (saved in a public-visibility",
                                     "Google Drive folder). Raw CSV data are cleaned and resampled to multi-monthly metrics using Python 3.",
                                     "Data and code were prepared by Alex Pakalniskis (pakalniskis@email.arizona.edu) during Summer 2019."])
    
    re_mean_dfsc_JFM = dfsc_mean_JFM.resample("Q-Mar").mean()
    re_stdDev_dfsc_JFM = dfsc_stdDev_JFM.resample("Q-Mar").mean()
    
    dfsc_mean_JFM_crop = re_mean_dfsc_JFM[re_mean_dfsc_JFM.index.month.isin([3])]
    dfsc_stdDev_JFM_crop = re_stdDev_dfsc_JFM[re_stdDev_dfsc_JFM.index.month.isin([3])]
    
    dfsc_mean_JFM_crop.index = dfsc_mean_JFM_crop.index.strftime("%Y")
    dfsc_stdDev_JFM_crop.index = dfsc_stdDev_JFM_crop.index.strftime("%Y")
    
    dfsc_mean_JFM_crop.columns = dfsc_mean_JFM_crop.columns + "_JFM"
    dfsc_stdDev_JFM_crop.columns = dfsc_stdDev_JFM_crop.columns + "_JFM"
            
    dfsc_mean_FMA = dfs_mean.copy()
    dfsc_stdDev_FMA = dfs_stdDev.copy()
    df_readme_FMA = pd.DataFrame(["This is a cleaned data set depicting longest dry stretch per month (days with precipitation < 0.1 mm)",
                                     "zonal statistics across Zambia's 101 administrative districts for",
                                     "the FMA agricultural growing sub-season. Daily precipitation data from UCSB Climate Hazards Group",
                                     "CHIRPS 2.0 were accessed using Google Earth Engine JavaScript API. Raster data are transformed to",
                                     "CSV files during zonal statistics processing (saved in a public-visibility",
                                     "Google Drive folder). Raw CSV data are cleaned and resampled to multi-monthly metrics using Python 3.",
                                     "Data and code were prepared by Alex Pakalniskis (pakalniskis@email.arizona.edu) during Summer 2019."])
    
    re_mean_dfsc_FMA = dfsc_mean_FMA.resample("Q-Apr").mean()
    re_stdDev_dfsc_FMA = dfsc_stdDev_FMA.resample("Q-Apr").mean()
    
    dfsc_mean_FMA_crop = re_mean_dfsc_FMA[re_mean_dfsc_FMA.index.month.isin([4])]
    dfsc_stdDev_FMA_crop = re_stdDev_dfsc_FMA[re_stdDev_dfsc_FMA.index.month.isin([4])]
    
    dfsc_mean_FMA_crop.index = dfsc_mean_FMA_crop.index.strftime("%Y")
    dfsc_stdDev_FMA_crop.index = dfsc_stdDev_FMA_crop.index.strftime("%Y")
    
    dfsc_mean_FMA_crop.columns = dfsc_mean_FMA_crop.columns + "_FMA"
    dfsc_stdDev_FMA_crop.columns = dfsc_stdDev_FMA_crop.columns + "_FMA"        
        
    with pd.ExcelWriter(output_file) as writer:
        dfs_mean.to_excel(writer, sheet_name='MEAN_monthly_longest_dry_stretch')
        dfs_stdDev.to_excel(writer, sheet_name='STDDEV_monthly_longest_dry_stretch')
        df_readme.to_excel(writer, sheet_name='README_monthly_longest_dry_stretch')
        
        dfsc_mean_NDJFMA.to_excel(writer, sheet_name='MEAN_NDJFMA_monthly_longest_dry_stretch')
        dfsc_stdDev_NDJFMA.to_excel(writer, sheet_name='STDDEV_NDJFMA_monthly_longest_dry_stretch')
        df_readme_NDJFMA.to_excel(writer, sheet_name='README_NDJFMA_monthly_longest_dry_stretch')
        
        dfsc_mean_NDJ_crop.to_excel(writer, sheet_name='MEAN_NDJ_monthly_longest_dry_stretch')
        dfsc_stdDev_NDJ_crop.to_excel(writer, sheet_name='STDDEV_NDJ_monthly_longest_dry_stretch')
        df_readme_NDJ.to_excel(writer, sheet_name='README_NDJ_monthly_longest_dry_stretch')
        
        dfsc_mean_DJF_crop.to_excel(writer, sheet_name='MEAN_DJF_monthly_longest_dry_stretch')
        dfsc_stdDev_DJF_crop.to_excel(writer, sheet_name='STDDEV_DJF_monthly_longest_dry_stretch')
        df_readme_DJF.to_excel(writer, sheet_name='README_DJF_monthly_longest_dry_stretch')
        
        dfsc_mean_JFM_crop.to_excel(writer, sheet_name='MEAN_JFM_monthly_longest_dry_stretch')
        dfsc_stdDev_JFM_crop.to_excel(writer, sheet_name='STDDEV_JFM_monthly_longest_dry_stretch')
        df_readme_JFM.to_excel(writer, sheet_name='README_JFM_monthly_longest_dry_stretch')
        
        dfsc_mean_FMA_crop.to_excel(writer, sheet_name='MEAN_FMA_monthly_longest_dry_stretch')
        dfsc_stdDev_FMA_crop.to_excel(writer, sheet_name='STDDEV_FMA_monthly_longest_dry_stretch')
        df_readme_FMA.to_excel(writer, sheet_name='README_FMA_monthly_longest_dry_stretch')
        
    print("Your multi-monthly metrics are ready at \n" + output_file + ". \nEnjoy!")
    
    files.download(output_file)

In [30]:
data_cleaning_monthly_longest_dry_spell(monthly_longest_dry_day_spell_csv, output_file)

/usr/local/lib/python3.6/dist-packages/openpyxl/workbook/child.py:102: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Your multi-monthly metrics are ready at 
administrative-district-monthly-longest-dry-stretch.xlsx. 
Enjoy!
